In [9]:
import os
import sys

from bids.grabbids import BIDSLayout
from bidst.transformers import ROITransformer
from bidst.transformers import SkullStrippingTransformer

from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import BET

from sklearn.pipeline import Pipeline


## To run this notebook you need to download data from the folloswing links and put it (with the same names) into the folder /bidst/test/

https://drive.google.com/drive/folders/0B6U5KQalulfAOXlHYkdIa25QbG8?usp=sharing

https://www.dropbox.com/sh/kh5vkp5s5n6eebh/AABejlvSqHV1HhFy91r9nrT8a?dl=0

# BIDS Transformers Example

This is simple example which shows two BIDS Transformers: first for ROI extraction and second for a skull stripping. 

In [18]:
project_root = os.path.abspath('bidst/tests/data/ds114')
roi = ROITransformer(label='B0',
                     t_min=0,
                     t_size=1,
                     basedir=project_root)

skull_str = SkullStrippingTransformer(basedir=project_root)

pp = Pipeline([('roi', roi),
               ('bet', skull_str)])

#temp = pp.fit_transform(X=[('01', 'test'),
#                  ('02', 'test'),
#                  ('02', 'retest'),
#                  ('03', 'retest')])

temp = pp.fit_transform([('01', 'retest'),
                  ('02', 'retest')])

2


In [16]:
roi.fit_transform([('01', 'retest'),
                  ('02', 'retest')])

['/home/deaddy/Repos/BIDSTransformers/bidst/tests/ds114/derivatives/ROITransformer/ses-retest/dwi/sub-01_ses-retest_dwi_label-label_label-BO.nii.gz',
 '/home/deaddy/Repos/BIDSTransformers/bidst/tests/ds114/derivatives/ROITransformer/ses-retest/dwi/sub-02_ses-retest_dwi_label-label_label-BO.nii.gz',
 '/home/deaddy/Repos/BIDSTransformers/bidst/tests/ds114/derivatives/ROITransformer/ses-retest/dwi/sub-01_ses-retest_dwi_label-label_label-BO.nii.gz',
 '/home/deaddy/Repos/BIDSTransformers/bidst/tests/ds114/derivatives/ROITransformer/ses-retest/dwi/sub-02_ses-retest_dwi_label-label_label-BO.nii.gz']

# Nipype Example

Типа можно сделать workflow, но здесь это избыточно.

## Defining files for preprocessing

In [ ]:
project_root = os.path.abspath('bidst/tests/ds114')
layout = BIDSLayout(project_root)
files = layout.get(extensions='dwi.nii.gz')
file_paths = [f.filename for f in files]
file_paths

In [ ]:
import nipype.pipeline.engine as pe


def get_sub_ses_type(file_path):
    dirname = os.path.dirname(file_path)
    sub_ses_type = dirname.split('/')[-3:]
    sub_ses_type = '/'.join(sub_ses_type)
    return sub_ses_type


def get_roi_filename(file_path):
    basename = os.path.basename(file_path)
    no_ext_name = os.path.splitext(basename)[0]
    no_ext_name = os.path.splitext(no_ext_name)[0]
    no_ext_name += '_label-label_roi'
    normal_name = no_ext_name + '.nii.gz'
    return normal_name


def get_bet_filename(file_path):
    basename = os.path.basename(file_path)
    no_ext_name = os.path.splitext(basename)[0]
    no_ext_name = os.path.splitext(no_ext_name)[0]
    no_ext_name += '_brain'
    normal_name = no_ext_name + '.nii.gz'
    return normal_name    

for file_path in file_paths:
    pipeline_name = 'roi_bet'
    sub_ses_type = get_sub_ses_type(file_path)
    roi_path = '{}/derivatives/{}/{}'.format(project_root, pipeline_name, sub_ses_type)
    if not os.path.exists(roi_path):
        os.makedirs(roi_path)
    roi_basename = get_roi_filename(file_path)
    roi_file_path = '{}/{}'.format(roi_path, roi_basename)
    fslroi = ExtractROI(in_file=file_path,
                        roi_file=roi_file_path,
                        t_min=0,
                        t_size=1)
    fslroi.run()
    bet_basename = get_bet_filename(file_path)
    bet_file_path = '{}/{}'.format(roi_path, bet_basename)
    fslbet = BET(in_file=roi_file_path,
                 out_file=bet_file_path)
    fslbet.run()
    print(roi_file_path)
    print(bet_file_path)